In [1]:
import os
from dotenv import load_dotenv

In [2]:
import uuid

# Generate a unique run ID for this experiment
run_uid = uuid.uuid4().hex[:6]

In [3]:
from langchain_benchmarks import clone_public_dataset, registry

In [4]:
registry = registry.filter(Type="RetrievalTask")
registry

Name,Type,Dataset ID,Description
LangChain Docs Q&A,RetrievalTask,452ccafc-18e1-4314-885b-edd735f17b9d,Questions and answers based on a snapshot of the LangChain python docs. The environment provides the documents and the retriever information. Each example is composed of a question and reference answer. Success is measured based on the accuracy of the answer relative to the reference answer. We also measure the faithfulness of the model's response relative to the retrieved documents (if any).
Semi-structured Reports,RetrievalTask,c47d9617-ab99-4d6e-a6e6-92b8daf85a7d,Questions and answers based on PDFs containing tables and charts. The task provides the raw documents as well as factory methods to easily index them and create a retriever. Each example is composed of a question and reference answer. Success is measured based on the accuracy of the answer relative to the reference answer. We also measure the faithfulness of the model's response relative to the retrieved documents (if any).


In [5]:
langchain_docs = registry["LangChain Docs Q&A"]
langchain_docs

Name,LangChain Docs Q&A
Type,RetrievalTask
Dataset ID,452ccafc-18e1-4314-885b-edd735f17b9d
Description,Questions and answers based on a snapshot of the LangChain python docs. The environment provides the documents and the retriever information. Each example is composed of a question and reference answer. Success is measured based on the accuracy of the answer relative to the reference answer. We also measure the faithfulness of the model's response relative to the retrieved documents (if any).
Retriever Factories,"basic, parent-doc, hyde"
Architecture Factories,conversational-retrieval-qa
get_docs,


In [6]:
#os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
load_dotenv()

clone_public_dataset(langchain_docs.dataset_id, dataset_name=langchain_docs.name)

Dataset LangChain Docs Q&A already exists. Skipping.
You can access the dataset at https://smith.langchain.com/o/cb5976d7-e8f5-5553-aca5-41aa75ce8690/datasets/cb68a322-4869-4e47-bc31-793a86e12074.


In [7]:
docs = list(langchain_docs.get_docs())
print(repr(docs[0])[:100] + "...")

Document(page_content="LangChain cookbook | 🦜️🔗 Langchain\n\n[Skip to main content](#docusaurus_skip...


In [8]:
len(docs)

2051

In [1]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores.chroma import Chroma

embeddings = HuggingFaceEmbeddings(
    model_name="thenlper/gte-base",
    # model_kwargs={"device": 0},  # Comment out to use CPU
)

vectorstore = Chroma(
    collection_name=f"lcbm-b-huggingface-gte-base",
    embedding_function=embeddings,
    persist_directory="./chromadb",
)

In [11]:
vectorstore.add_documents([docs[0]])

['a73502be-89e3-11ee-99ce-5e5044592b71']

In [12]:
vectorstore.add_documents(docs)
retriever = vectorstore.as_retriever(search_kwargs={"k": 6})

In [9]:
langchain_docs.name

'LangChain Docs Q&A'

In [14]:
x = list(langchain_docs.get_docs())

In [16]:
x[0]

Document(page_content="LangChain cookbook | 🦜️🔗 Langchain\n\n[Skip to main content](#docusaurus_skipToContent_fallback)# LangChain cookbook\n\nExample code for building applications with LangChain, with an emphasis on more applied and end-to-end examples than contained in the [main documentation](https://python.langchain.com).\n\n| Notebook | Description |\n| ---- | ---- |\n| LLaMA2_sql_chat.ipynb | Build a chat application that interacts with a SQL database using an open source llm (llama2), specifically demonstrated on an SQLite database containing rosters. |\n| Semi_Structured_RAG.ipynb | Perform retrieval-augmented generation (rag) on documents with semi-structured data, including text and tables, using unstructured for parsing, multi-vector retriever for storing, and lcel for implementing chains. |\n| Semi_structured_and_multi_moda... | Perform retrieval-augmented generation (rag) on documents with semi-structured data and images, using unstructured for parsing, multi-vector retri

In [29]:
import json



def read_jsonl_file(file_path):
    size = 0
    data = []
    
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            # Load each line as a JSON object
            print(line,'\n')
            json_object = json.loads(line.strip())
            data.append(json_object)
            size += num_tokens(json_object['input'])
            size += num_tokens(json_object['ideal'])
            
    return data,size 

# Example usage
file_path = 'eval.jsonl'
jsonl_data,size = read_jsonl_file(file_path)

# Now, jsonl_data contains a list of dictionaries, each representing a JSON object from the file
print(jsonl_data)
print(size)


{"input": "in code, how can i add a system message at the end of the conversation history to influence the output of the llm", "ideal": "\n```python\nfrom langchain.prompts import ChatPromptTemplate, MessagesPlaceholder\n\nprompt_template = ChatPromptTemplate.from_messages(\n    [\n        (\"human\", \"{user_input}\"),\n        MessagesPlaceholder(variable_name=\"chat_history\")\n        (\"system\", \"New System Instruction here\"),\n    ]\n)\n```\nWhen formatted, it will generate 1 human/user message and 1 system message, as well as messages for the whole conversation history.\n```python\nprompt_template.format_messages(user_input=\"Hello\", chat_history=[])\n```\n[HumanMessage(content='Hello', additional_kwargs={}, example=False),\n SystemMessage(content='New System Instruction here', additional_kwargs={})]\n"}
 

{"input": "how to use enums in Tools that will be used by functions agent?", "ideal": "If you're defining a dynamic tool (either using the @tool decorator or the `.from_f

In [21]:
f


```python
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt_template = ChatPromptTemplate.from_messages(
    [
        ("human", "{user_input}"),
        MessagesPlaceholder(variable_name="chat_history")
        ("system", "New System Instruction here"),
    ]
)
```
When formatted, it will generate 1 human/user message and 1 system message, as well as messages for the whole conversation history.
```python
prompt_template.format_messages(user_input="Hello", chat_history=[])
```
[HumanMessage(content='Hello', additional_kwargs={}, example=False),
 SystemMessage(content='New System Instruction here', additional_kwargs={})]



In [27]:
import openai  # for generating embeddings
import pandas as pd  # for DataFrames to store article sections and embeddings
import re  # for cutting <ref> links out of Wikipedia articles
import tiktoken  # for counting tokens

GPT_MODEL = "gpt-4"  # only matters insofar as it selects which tokenizer to use


def num_tokens(text: str, model: str = GPT_MODEL) -> int:
    """Return the number of tokens in a string."""
    encoding = tiktoken.encoding_for_model(model)
    return len(encoding.encode(text))


In [28]:
num_tokens("hello this is me manish bhatta")

9

In [32]:
ans = size/1000

price per evaluation

In [34]:
size

8523

In [33]:
ans* 0.03

0.25569

In [35]:
1000*5 / 4

1250.0

In [37]:
1250*86 / 1000

107.5

In [38]:
107.5 * 0.01

1.075